In [ ]:
groq_api_key=r"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"
URI="neo4j+s://063cc342.databases.neo4j.io"
Uname="neo4j"
passw="zoy6gdX9KQyz1h5V3LntJ74IjG6A_hTML83GTQv6LVw"
DATABASE="neo4j"
INST="063cc342"
INAME="paid-1"


In [136]:
from google.colab import userdata

groq_api_key=userdata.get('groq_api_key')
URI=userdata.get('URI')
Uname=userdata.get('Uname')
passw=userdata.get('passw')
DATABASE=userdata.get('DATABASE')
INST=userdata.get('INST')
INAME=userdata.get('INAME')

In [ ]:
! pip install  langchain_core
! pip install langchain_experimental
! pip install  neo4j langchain_graphdb
! pip install langchain_groq
! pip install neo4j
from neo4j import GraphDatabase, Query

In [ ]:

# aura db is cloud based graph database
from langchain.graphs import Neo4jGraph
graph = Neo4jGraph(
    url="neo4j+s://063cc342.databases.neo4j.io",
    username="neo4j",
    password=passw,
)
# tips to connect, use https rather than neo4j+s: for protocol and provide password for it too username must like be neo4j in curreent case

"""
Three types of searches are allowe dover neo4j:
1- range:less tha euql or more tha type search
2- text: exact/prefix/sufix serach
3- fuzzy serach
"""


/tmp/ipython-input-3724613611.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


'\nThree types of searches are allowe dover neo4j:\n1- range:less tha euql or more tha type search\n2- text: exact/prefix/sufix serach\n3- fuzzy serach\n'

In [ ]:
try:
    driver = GraphDatabase.driver(URI, auth=(Uname, passw))
    driver.verify_connectivity()
    print("Connection to Neo4j successful!")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")
    # In a real application, you might want more robust error handling
    exit()

Connection to Neo4j successful!


In [111]:

from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
# it will  convert  doc with its metadata , when save over vector db, metadata will also get save over vector db. such as the source file name, the page number, or the creation date. This metadata is crucial for advanced retrieval and filtering operations., for both metadata will alo get sqave
text="""
My name is Adarsh Kumar Gupta, and I am an AI/ML Engineer with 1.8 years of hands-on experience at Accenture, specializing in the practical deployment of artificial intelligence and machine learning solutions in cloud-native environments.
I know people with name adarsh0, ada1rsh and 2adarsh  who works at zs company for more than 3 years.
Sir Alex Ferguson, Alex Johnson and Sarah Smith also works at Accenture
I just bought a new Dell laptop, model XPS 13. It's great for programming, but the battery life is a little short. I am a software engineer. - by A. Jones
I absolutely love my new laptop from Dell. It is the best one I've ever owned. The model I have is an XPS 13. - by Alex Jones
I'm a freelancer and needed a fast computer. The Dell XPS 13 is a great machine for my work. - by Alexander Jones
Manchester United Football Club is a professional football club based in Old Trafford, Greater Manchester, England. The club was founded as Newton Heath LYR Football Club in 1878. They play at Old Trafford, which is a stadium.
Sir Alex Ferguson, Alex Johnson and Sarah Smith, a Scottish football manager, managed Manchester United from 1986 to 2013. He is widely regarded as one of the greatest managers of all time.
 all 3 works at accenture"""
# documents=[Document(page_content=text, metadata={"about":"we r using this data without and text splittter, for big doc to convert into chunks off documents, kindly use text splutter functionality of langchain"})]


additional_instructions="""
     You are an expert at extracting entities and their relationships from text. Your task is to extract information and transform it into a graph document.

     Instructions:
     1.  Extract all nodes (entities) and relationships from the provided text.
     2.  Each node must have a 'type' and 'id'.
     3.  Crucially, extract all relevant properties for each node. For a 'Person', this should include 'name', 'job_title', and 'experience'. For a 'Product', include 'name' and 'model'. For a 'Sportsteam', include 'name' and 'founded_year'.
     4.  For relationships, include 'source', 'target', and 'type'.
     5.  If a relationship has properties (e.g., years of a job), extract them as well.
     6.  Do not hallucinate or create nodes or properties that are not explicitly mentioned in the text.

     Example: if u are reruring node then , it should have id, type and *properties*.
     Kindly return properties which will must hold enetities which are related to node. consider below data to decide what values could be in properties:

      if you are conveting this sentence into nodes then , jane doe is a data scientits. then propties of this node must be like this
       "nodes":  will have  attributes like

         output format:(  "id": "Jane Doe",
           "type": "Person",
           "properties": {{
             "name": "Jane Doe",
             "job_title": "Data Scientist"}}
             )



     Schema to follow:
     - Nodes:
       - Person: properties should include 'name', 'job_title', 'experience'
       - Organization: properties should include 'name'
       - Product: properties should include 'name', 'model'
       - Sportsteam: properties should include 'name', 'founded_year'
       - Stadium: properties should include 'name'
       - Jobtitle: properties should include 'name'
     - Relationships:
       - WORKS_FOR (Person -> Organization)
       - OCCUPATION (Person -> Jobtitle)
       - PURCHASED (Person -> Product)
       - PURCHASED_FROM (Person -> Organization)
       - PLAYS_AT (Sportsteam -> Stadium)
       - MANAGED (Person -> Sportsteam): properties can include 'start_year', 'end_year'
       - ORIGINAL_NAME (Sportsteam -> Sportsteam): properties can include 'founded_year'

    Important: return something in ur properties of your node
     """

custom_prompt = ChatPromptTemplate.from_messages([("system:"+str(additional_instructions)), ("human", "Extract the graph from the following text:\n{input}")])
documents=[Document(page_content=text)]
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm.invoke("hi")

additional_instructions=""
from langchain_experimental.graph_transformers import LLMGraphTransformer
# llm_transformer=LLMGraphTransformer(llm=llm, additional_instructions=additional_instructions)
llm_transformer=LLMGraphTransformer(llm=llm, prompt=custom_prompt)
# we can give prompts = some _new_prompt but it will consider new one and start ognoring otthr prompts, so give as additonal_info

graph_documents=llm_transformer.convert_to_graph_documents(documents)
# so it will give output as listing of all nodes:(node1...node_n), then tell about relationship:[relationship btw all the nodes]. also other granular information will become become properties of node, it may not be a node

graph_documents[0].nodes, graph_documents[0].relationships

([Node(id='Adarsh Kumar Gupta', type='Person', properties={}),
  Node(id='Adarsh0', type='Person', properties={}),
  Node(id='Ada1Rsh', type='Person', properties={}),
  Node(id='2Adarsh', type='Person', properties={}),
  Node(id='Sir Alex Ferguson', type='Person', properties={}),
  Node(id='Alex Johnson', type='Person', properties={}),
  Node(id='Sarah Smith', type='Person', properties={}),
  Node(id='A. Jones', type='Person', properties={}),
  Node(id='Alex Jones', type='Person', properties={}),
  Node(id='Alexander Jones', type='Person', properties={}),
  Node(id='Accenture', type='Organization', properties={}),
  Node(id='Zs Company', type='Organization', properties={}),
  Node(id='Dell', type='Organization', properties={}),
  Node(id='Manchester United Football Club', type='Sportsteam', properties={}),
  Node(id='Old Trafford', type='Stadium', properties={}),
  Node(id='Xps 13', type='Product', properties={})],
 [Relationship(source=Node(id='Adarsh Kumar Gupta', type='Person', prop

In [112]:
graph_documents

[GraphDocument(nodes=[Node(id='Adarsh Kumar Gupta', type='Person', properties={}), Node(id='Adarsh0', type='Person', properties={}), Node(id='Ada1Rsh', type='Person', properties={}), Node(id='2Adarsh', type='Person', properties={}), Node(id='Sir Alex Ferguson', type='Person', properties={}), Node(id='Alex Johnson', type='Person', properties={}), Node(id='Sarah Smith', type='Person', properties={}), Node(id='A. Jones', type='Person', properties={}), Node(id='Alex Jones', type='Person', properties={}), Node(id='Alexander Jones', type='Person', properties={}), Node(id='Accenture', type='Organization', properties={}), Node(id='Zs Company', type='Organization', properties={}), Node(id='Dell', type='Organization', properties={}), Node(id='Manchester United Football Club', type='Sportsteam', properties={}), Node(id='Old Trafford', type='Stadium', properties={}), Node(id='Xps 13', type='Product', properties={})], relationships=[Relationship(source=Node(id='Adarsh Kumar Gupta', type='Person', p

In [113]:
graph.add_graph_documents(graph_documents)

In [114]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True, allow_dangerous_requests=True, validate_cypher=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x789766fede80>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7897532de4e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x789747871430>, model_name='Gemma2-9b-I

In [115]:
"""
It is returning null bcz actual data in neo4j is Adarsh Kumar Gupta anot adarsh kumar gupta, so we will use fuzzy here to get similar matches using apache lucine.
Steps:
Create an index : full text search # fuzzy
serach ur terms over there and get similar matches.
use that data and find it over full cluster.
"""

response=chain.invoke({"query":"who is person whose id is  Adarsh0"})

response





> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id:'Adarsh0'}) RETURN p

Full Context:
[{'p': {'id': 'Adarsh0'}}]

> Finished chain.


{'query': 'who is person whose id is  Adarsh0',
 'result': 'Adarsh0 is a person.  \n'}

# Fuzzy serach over Graph DB

In [129]:
index0="""CREATE FULLTEXT INDEX index_name IF NOT EXISTS
FOR (n:Person) ON EACH [n.id]

"""
# created a index on person node

# creating  index over Person node to avoild full nodes a=search for fuzzy
q1_out=driver.execute_query(index0)
q1_out

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x789747a933b0>, keys=[])

In [131]:
# now testing index using  fuzzy logic input

q1="""CALL db.index.fulltext.queryNodes('index_name', 'adarsh~0.01')
YIELD node, score
RETURN node.id , score
ORDER BY score DESC
"""

# it will split workds and search which node contain max terms and score accordingly, same as open search
# ~, fuzzy will take laventeain distance

q1_out=driver.execute_query(q1)
q1_out

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x789747678560>, keys=['node.id', 'score'])

In [132]:
# withput fuzzy
q2="""CALL db.index.fulltext.queryNodes('index_name', 'alen')
YIELD node, score
RETURN node.id, score
ORDER BY score DESC
"""

# with fuzzy
q22="""CALL db.index.fulltext.queryNodes('index_name', 'alen~')
YIELD node, score
RETURN node.id, score
ORDER BY score DESC
"""

q2_out=driver.execute_query(q2)
q22_out=driver.execute_query(q22)
print("q2_out",q2_out)
print("------------------")
print("q22_out",q22_out)


q2_out EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x78974767b290>, keys=['node.id', 'score'])
------------------
q22_out EagerResult(records=[<Record node.id='Alex Johnson' score=0.38215771317481995>, <Record node.id='Alex Jones' score=0.38215771317481995>, <Record node.id='Sir Alex Ferguson' score=0.3156312108039856>], summary=<neo4j._work.summary.ResultSummary object at 0x789747a914f0>, keys=['node.id', 'score'])


In [133]:
q3="""
match (p:Person)
return p
"""

q3_out=driver.execute_query(q3)
q3_out

EagerResult(records=[<Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:0' labels=frozenset({'Person'}) properties={'city': 'London', 'name': 'Alex Johnson', 'id': 2, 'age': 25}>>, <Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:1' labels=frozenset({'Person'}) properties={'city': 'New York', 'name': 'Sarah Smith', 'id': 3, 'age': 35}>>, <Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:3' labels=frozenset({'Person'}) properties={'id': 'Adarsh Kumar Gupta'}>>, <Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:4' labels=frozenset({'Person'}) properties={'id': 'Adarsh0'}>>, <Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:5' labels=frozenset({'Person'}) properties={'id': 'Ada1Rsh'}>>, <Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:6' labels=frozenset({'Person'}) properties={'id': '2Adarsh'}>>, <Record p=<Node element_id='4:56c61591-99cb-440c-ae9e-ac7018f31a4a:7' labels=frozenset({'Pe

# Loading data rom comma seprated file


In [118]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
import pandas as pd
df1=pd.read_csv(r"/content/grpah_db_csv1 - Sheet1.csv")
df2=pd.read_csv(r"/content/grpah_db_csv1 - Sheet2.csv")

In [120]:
df1.head()


,person_id,name,age,city
0,1,Adarsh Kumar Gupta,30,Bangalore
1,2,Alex Johnson,25,London
2,3,Sarah Smith,35,New York


In [121]:
# nodes creation : merge: it will update i not property node is existing else, no dublicate generation like create command
final_query=""
for i, d in df1.iterrows():
  # print(d.keys())
  employee_query = """
    MERGE ({nn}:Person {{id: {id}}})
    ON CREATE SET
        {nn}.name = '{name}',
        {nn}.age = {age},
        {nn}.city = '{city}'
    """.format(id=d["person_id"],name=d["name"],city=d["city"], age=d["age"], nn=str(d["name"].split(" ")[0]))
  print(employee_query)
  final_query+=employee_query




    MERGE (Adarsh:Person {id: 1})
    ON CREATE SET
        Adarsh.name = 'Adarsh Kumar Gupta',
        Adarsh.age = 30,
        Adarsh.city = 'Bangalore'
    

    MERGE (Alex:Person {id: 2})
    ON CREATE SET
        Alex.name = 'Alex Johnson',
        Alex.age = 25,
        Alex.city = 'London'
    

    MERGE (Sarah:Person {id: 3})
    ON CREATE SET
        Sarah.name = 'Sarah Smith',
        Sarah.age = 35,
        Sarah.city = 'New York'
    


In [122]:
q1_out=driver.execute_query(final_query)
q1_out

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x789747ac75c0>, keys=[])

In [123]:
df2.head()

,source_person_id,target_person_id,relationship_type,since_year
0,1,2,KNOWS,2020
1,2,3,WORKS_WITH,2022
2,1,3,FRIENDS,2018


In [125]:
# nodes creation : merge: it will update i not property node is existing else, no dublicate generation like create command
final_query=""
for i, d in df2.iterrows():
  # print(d.keys())
  employee_query = """
    match ({nn}:Person {{id: {nn1}}})
    match ({mm}:Person {{id: {mm1}}})
    merge({nn})-[{rel_type}:rel_type]->({mm})
    on  create set
    {rel_type}.relationship_type='{relationship_type}',
    {rel_type}.since_year={since_year}

    """.format(nn="a"+str(d["source_person_id"]),mm="a"+str(d["target_person_id"]), nn1=d["source_person_id"],
               mm1=d["target_person_id"],relationship_type=str(d["relationship_type"]), since_year=d["since_year"],rel_type=str(d["relationship_type"])+"aa")
  print(employee_query)
  final_query+=employee_query
  q1_out=driver.execute_query(employee_query)
  print("insert query output: ",q1_out)




    match (a1:Person {id: 1})
    match (a2:Person {id: 2})
    merge(a1)-[KNOWSaa:rel_type]->(a2)
    on  create set 
    KNOWSaa.relationship_type='KNOWS',
    KNOWSaa.since_year=2020
    
    
insert query output:  EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x789747a92390>, keys=[])

    match (a2:Person {id: 2})
    match (a3:Person {id: 3})
    merge(a2)-[WORKS_WITHaa:rel_type]->(a3)
    on  create set 
    WORKS_WITHaa.relationship_type='WORKS_WITH',
    WORKS_WITHaa.since_year=2022
    
    
insert query output:  EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x789747a92ae0>, keys=[])

    match (a1:Person {id: 1})
    match (a3:Person {id: 3})
    merge(a1)-[FRIENDSaa:rel_type]->(a3)
    on  create set 
    FRIENDSaa.relationship_type='FRIENDS',
    FRIENDSaa.since_year=2018
    
    
insert query output:  EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x789747a90bc0>, keys=[])
